In [1]:
from biopandas.pdb import PandasPdb
from tqdm.notebook import tqdm
import glob

import random
import urllib
import pickle
random.seed(52)

from protein_tools import *

### initialise PyRosetta
pr.init('-ignore_unrecognized_res -ignore_zero_occupancy -mute all -holes:dalphaball /work/lpdi/bin/rosetta_06_2021/main/source/external/DAlpahBall/DAlphaBall.gcc -corrections::beta_nov16 true -relax:default_repeats 1')

PyRosetta-4 2023 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python310.Release 2023.35+release.23439d33534e3f106f2ad301c8c3c56013ca8471 2023-08-30T15:39:05] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


# Sample natural PPIs from CATH

In [ ]:
# native pdb codes are in cath-domain-list-S100.txt, every line has this format:
#1oaiA00     1    10     8    10     1     1     1     1     1    59 1.000
# pdb code is the first 4 characters
# read all pdb codes from this file

native_pdb_codes = []
with open('cath-domain-list-S100.txt') as f:
    for line in f:
        native_pdb_codes.append(line[:4])

# remove duplicates
native_pdb_codes = list(set(native_pdb_codes))

# subselect random 1000
random.shuffle(native_pdb_codes)
native_pdb_codes = native_pdb_codes[:1029]

for pdb_code in native_pdb_codes:
    url = 'https://files.rcsb.org/download/' + pdb_code + '.pdb'
    filename = 'native_domain_database/' + pdb_code + '.pdb'
    if os.path.isfile(filename):
        continue
    print('Downloading ' + pdb_code + ' from ' + url + ' to ' + filename)
    try:
        urllib.request.urlretrieve(url, filename)
    except:
        print('Error downloading ' + pdb_code + ' from ' + url)

# Calculate the native PPI interface scores

In [3]:
for pdb in tqdm(glob.glob('native_binder_database_clean/*.pdb')):
    name = pdb.split('/')[-1].replace('.pdb','')
    print(name)
    pdb_file = pdb
    
    chain_A = pdb.split('/')[-1].split('_')[1]
    chain_B = pdb.split('/')[-1].split('_')[2].replace('.pdb', '')
    
    # randomly select weather to use chain A or chain B
    random_number = random.uniform(0, 1)
    if random_number < 0.5:
        chain = chain_A
        if len(chain_A) != 1:
            chain = chain_B
    else:
        chain = chain_B

    try:
        interface_scores, interface_AA, interface_residues_pdb_ids_str = score_interface(pdb_file, chain)
        total_count, helix_count, sheet_count, loop_count = obtain_sse_content_interface(pdb_file, chain)

        pose = pr.pose_from_pdb(pdb_file)
        interface_scores['interface_bb_hbonds'] = count_interface_bb_hbonds(pose)
        chain1_interface_residues, chain2_interface_residues = get_interface_residues(pose, chain_A, chain_B)
        interface_scores['interface_hydrophobicity'] = calculate_hydrophobicity(pose, chain1_interface_residues)
        interface_scores['loop_perc'] = (loop_count/total_count)*100
        interface_scores['helix_perc'] = (helix_count/total_count)*100
        interface_scores['beta_perc'] = (sheet_count/total_count)*100

        with open('native_PPI_interface_scores/{name}.pickle'.format(name=name), 'wb') as handle:
            pickle.dump(interface_scores, handle, protocol=pickle.HIGHEST_PROTOCOL)
    except:
        print('error with:', name, 'skipping...')

  0%|          | 0/926 [00:00<?, ?it/s]

1GCQ_A_B
1L4D_A_B
3I2B_A_B
3QWQ_A_B
2HVY_A_B
2ABZ_A_B
2FP7_A_B
1TM3_A_B
1AVX_A_B
3QC8_A_B
3WDG_A_B
3EPZ_A_B
3BCP_A_B
1T0F_A_B
3K4W_A_B
2YVJ_A_B
3VPJ_A_B
3F74_A_B
3Q87_A_B
1WDX_A_B
2P04_A_B
3HRD_A_B
1X1X_A_B
2GJV_A_B
5D1K_A_B
4TQ0_A_B
3DQQ_A_B
3NRJ_A_B
3LQV_A_B
1MBY_A_B
2FTM_A_B
2ZXW_A_B
2IDO_A_B
1X1Y_A_B
3GRW_A_B
3IAS_A_B
3I84_A_B
1FGL_A_B
4AYI_A_B
1XWD_A_B
1JBU_A_B
3LM1_A_B
1VGO_A_B
3E38_A_B
2OS7_A_B
3GQH_A_B
3RDZ_A_B
1VS3_A_B
1DJS_A_B
1UP6_A_B
3HO5_A_B
3BIW_A_B
3QQ8_A_B
1W1I_A_B
2DP4_A_B
1B27_A_B
2D10_A_B
1DML_A_B
2HZM_A_B
2ZSU_A_B
5F3X_A_B
2A6P_A_B
2DG0_A_B
4AFZ_A_B
3KW5_A_B
2OUL_A_B
4U30_A_B
3JVZ_A_B
3TG9_A_B
1KL8_A_B
3CJX_A_B
3GNJ_A_B
1EPT_A_B
3B01_A_B
1H1V_A_B
3LRJ_A_B
3RBQ_A_B
3MAL_A_B
4B1Y_A_B
3PPE_A_B
5B77_A_B
1VHJ_A_B
1ZH8_A_B
3QWN_A_B
2J12_A_B
3DSN_A_B
1Z3G_A_B
2FPE_A_B
3D1E_A_B
2P35_A_B
2A74_A_B
4CMM_A_B
1SUW_A_B
1GO4_A_B
1ZR0_A_B
1JKG_A_B
5B78_A_B
3FK9_A_B
4UDM_A_B
1GT7_A_B
3H35_A_B
3UI2_A_B
2JJT_A_B
4ZQU_A_B
1U20_A_B
2GD4_A_B
2VJF_A_B
1EJA_A_B
3CGY_A_B
2WQZ_A_B
1TM7_A_B
1

/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Polypeptide.py:144: BiopythonDeprecationWarning: 'three_to_one' will be deprecated in a future release of Biopython in favor of 'Bio.PDB.Polypeptide.protein_letters_3to1'.
  warnings.warn(
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'S' for Atom (name=SD) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'C' for Atom (name=CE) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/P

error with: 1UK4_A_B skipping...
3BGL_A_B
2I0B_A_B
4QZV_A_B
2NU1_A_B
2HJ1_A_B
3BRD_A_B
4K1R_A_B
1QB3_A_B
1S4C_A_B
4IOP_A_B
2W1T_A_B
3O34_A_B
4NZL_A_B



ERROR: Error in core::scoring::methods::RamaPreProEnergy::residue_pair_energy(): The RamaPrePro term is incompatible with cyclic dipeptides (as is most of the rest of Rosetta).
ERROR:: Exit from: /home/benchmark/rosetta/source/src/core/energy_methods/RamaPreProEnergy.cc line: 127


...

error with: 2HDP_A_B skipping...
3E1Z_A_B
2G2W_A_B
3NCT_A_B
1EWY_A_B
1JYI_A_B
2FJU_A_B
2BMA_A_B
3FHC_A_B
1O9Y_A_B
2A5Z_A_B
2O8Q_A_B
3TL8_A_B
1T8U_A_B
4KBB_A_B
1DFJ_A_B
3KLQ_A_B
3GBU_A_B
3CE9_A_B
5DMJ_A_B
2YCH_A_B
3U02_A_B
4AFQ_A_B
2G2U_A_B
4AN7_A_B
1G60_A_B
1R7A_A_B
3BRC_A_B
4ILW_A_B
3ME4_A_B
4A94_A_B
4GH7_A_B
1GL1_A_B
4HDO_A_B
2PUY_A_B
3B93_A_B
2O9Q_A_B
1UWG_A_B
3D5N_A_B
2PMV_A_B
2NZ1_A_B
2R0K_A_B
1VG9_A_B
1LQM_A_B
2IWP_A_B
2NXM_A_B
3KMT_A_B
2R5O_A_B
2G6V_A_B
3HTR_A_B
1UNN_A_B
3GZE_A_B
2W80_A_B


/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Polypeptide.py:144: BiopythonDeprecationWarning: 'three_to_one' will be deprecated in a future release of Biopython in favor of 'Bio.PDB.Polypeptide.protein_letters_3to1'.
  warnings.warn(


error with: 2W80_A_B skipping...
1NQ9_A_B
3I5V_A_B
4LQW_A_B
2WQ4_A_B
4F0A_A_B
1VH4_A_B
2C1W_A_B


/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Polypeptide.py:144: BiopythonDeprecationWarning: 'three_to_one' will be deprecated in a future release of Biopython in favor of 'Bio.PDB.Polypeptide.protein_letters_3to1'.
  warnings.warn(


error with: 2C1W_A_B skipping...
3HPN_A_B
2HTB_A_B
1F2U_A_B
4BWQ_A_B
1JZO_A_B
1NP6_A_B
2ZNV_A_B


In [2]:
for pdb in tqdm(glob.glob('de_novo_binder_database_clean/*.pdb')):
    name = pdb.split('/')[-1].replace('.pdb','')
    print(name)
    pdb_file = pdb
    
    chain_A = pdb.split('/')[-1].split('_')[1]
    chain_B = pdb.split('/')[-1].split('_')[2].replace('.pdb', '')
    
    chain = chain_B

    interface_scores, interface_AA, interface_residues_pdb_ids_str = score_interface(pdb_file, chain)
    total_count, helix_count, sheet_count, loop_count = obtain_sse_content_interface(pdb_file, chain)

    pose = pr.pose_from_pdb(pdb_file)
    interface_scores['interface_bb_hbonds'] = count_interface_bb_hbonds(pose)
    chain1_interface_residues, chain2_interface_residues = get_interface_residues(pose, chain_A, chain_B)
    interface_scores['interface_hydrophobicity'] = calculate_hydrophobicity(pose, chain1_interface_residues)
    interface_scores['loop_perc'] = (loop_count/total_count)*100
    interface_scores['helix_perc'] = (helix_count/total_count)*100
    interface_scores['beta_perc'] = (sheet_count/total_count)*100

    with open('de_novo_PPI_interface_scores/{name}.pickle'.format(name=name), 'wb') as handle:
        pickle.dump(interface_scores, handle, protocol=pickle.HIGHEST_PROTOCOL)

  0%|          | 0/17 [00:00<?, ?it/s]

7zrv_A_B


/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Polypeptide.py:144: BiopythonDeprecationWarning: 'three_to_one' will be deprecated in a future release of Biopython in favor of 'Bio.PDB.Polypeptide.protein_letters_3to1'.
  warnings.warn(
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'C' for Atom (name=C) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PD

7tyd_A_B


/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Polypeptide.py:144: BiopythonDeprecationWarning: 'three_to_one' will be deprecated in a future release of Biopython in favor of 'Bio.PDB.Polypeptide.protein_letters_3to1'.
  warnings.warn(


...

/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Polypeptide.py:144: BiopythonDeprecationWarning: 'three_to_one' will be deprecated in a future release of Biopython in favor of 'Bio.PDB.Polypeptide.protein_letters_3to1'.
  warnings.warn(
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/P

7xad_A_B


/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Polypeptide.py:144: BiopythonDeprecationWarning: 'three_to_one' will be deprecated in a future release of Biopython in favor of 'Bio.PDB.Polypeptide.protein_letters_3to1'.
  warnings.warn(
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'C' for Atom (name=C) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PD

7rdh_A_B


/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Polypeptide.py:144: BiopythonDeprecationWarning: 'three_to_one' will be deprecated in a future release of Biopython in favor of 'Bio.PDB.Polypeptide.protein_letters_3to1'.
  warnings.warn(
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'S' for Atom (name=SG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'C' for Atom (name=CG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'N' for Atom (name=ND2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/goverde/miniconda3/envs/ColabDesign/lib/python3.10/site-packages/Bio